In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [69]:
from github import Github
from github import InputGitTreeElement
from datetime import datetime

# GET DATA

## General functions

In [10]:
def get_CREC_packages(pageSize, startDate= '2018-07-01T00:00:00'): #from collections
    apiKey = 'RjhPNKjGtGp1AbN4Ilb1jbrhCO9IBvqqLvRKsA1e'
    api_url = f'https://api.govinfo.gov/collections/CREC/{startDate}Z?offset=0&pageSize={pageSize}&api_key={apiKey}'
    data = requests.get(api_url).json()
    data = data['packages']
    return data

def ## Endpoints (TBD)get_CREC_summary(packageLink):   #packageLink retrieved from package table
    apiKey = 'RjhPNKjGtGp1AbN4Ilb1jbrhCO9IBvqqLvRKsA1e'
    url = packageLink + f'?api_key={apiKey}'
    record = requests.get(url).json()
    return record #one record each time
    
def get_CREC_granules(granulesLink):  # granulesLink retrieved from the summary table, get_CREC_summary(packageLink)['granulesLink']
    apiKey = 'RjhPNKjGtGp1AbN4Ilb1jbrhCO9IBvqqLvRKsA1e'
    url = granulesLink + f'&api_key={apiKey}'
    data = requests.get(url).json()
    count = data['count']
    granules = data['granules']
    return count,granules

# General Version: 
def get_CREC_granuleInfo(granuleLink): # granuleLink retrieved from the granule table
    apiKey = 'RjhPNKjGtGp1AbN4Ilb1jbrhCO9IBvqqLvRKsA1e'
    url = granuleLink + f'?api_key={apiKey}'
    data = requests.get(url).json()
    return data

def txt_download(granule_json):
    apiKey = 'RjhPNKjGtGp1AbN4Ilb1jbrhCO9IBvqqLvRKsA1e'
    url = granule_json['download']['txtLink'] + f'?api_key={apiKey}'
    raw_text = requests.get(url).text
    
    soup = BeautifulSoup(raw_text)
    text_body = soup.find('body').get_text()
    return text_body

## Endpoints (TBD)

In [ ]:
class Granule_Titles():
    def __init__(self,granules_data):
        self.all_granules = granules_data
        self.S_H_E_granules = [i for i in granules_data if i['granuleClass'] != 'DAILYDIGEST']
        
    def S_H_E_titles_txt(self):
        titles_txt = ''
        for granule in self.S_H_E_granules:
            granule_title = granule['title']
            titles_txt += granule_title
            titles_txt += '\n'
        return titles_txt
    
    def S_H_E_titles_df(self):
        titles_df = pd.DataFrame()
        titles_df['title'] = [i['title'] for i in self.all_senate]
        titles_df['granuleId'] = [i['granuleId'] for i in self.all_senate]
        titles_df['granuleClass'] = [i['granuleClass'] for i in self.all_senate]
        return titles_df

In [648]:
class DailyDigest():
    def __init__(self,granules_data):
        self.all_granules = granules_data
        self.all_digests = [i for i in granules_data if i['granuleClass'] == 'DAILYDIGEST']
        self.today_digests = self.all_digests[2:]
        
    #def get_titles():
    #def get_class():

    def today_digest_text(self):
        today_text = ''
        for record in self.today_digests:
            link = record['granuleLink']
            dailyDigest_json = get_CREC_granuleInfo(link)
            text_record = txt_download(dailyDigest_json)
            today_text += text_record
        return today_text
    
    def today_digest_df(self):
        today_df = pd.DataFrame()
        today_df['title'] = [i['title'] for i in self.today_digests]
        content = []
        for record in self.today_digests:
            link = record['granuleLink']
            dailyDigest_json = get_CREC_granuleInfo(link)
            text_record = txt_download(dailyDigest_json)
            content.append(text_record)
        today_df['content'] = content
        return today_df


In [ ]:
class Senate_House():
    def __init__(self,granules_data):
        self.all_granules = granules_data
        self.all_senate = [i for i in granules_data if i['granuleClass'] == 'SENATE']
        self.all_house = [i for i in granules_data if i['granuleClass'] == 'HOUSE']
    
    def today_Senate_df(self): 
        today_df = pd.DataFrame()
        today_df['title'] = [i['title'] for i in self.all_senate]
        today_df['granuleId'] = [i['granuleId'] for i in self.all_senate]
        content = []
        for record in self.today_digests:
            link = record['granuleLink']
            dailyDigest_json = get_CREC_granuleInfo(link)
            text_record = txt_download(dailyDigest_json)
            content.append(text_record)
        today_df['content'] = content
        return today_df        
        
    def today_House_df(self): 
        today_df = pd.DataFrame()
        today_df['title'] = [i['title'] for i in self.all_house]
        today_df['granuleId'] = [i['granuleId'] for i in self.all_house]
        content = []
        for record in self.today_digests:
            link = record['granuleLink']
            dailyDigest_json = get_CREC_granuleInfo(link)
            text_record = txt_download(dailyDigest_json)
            content.append(text_record)
        today_df['content'] = content
        return today_df        
    

# EXPORT DATA

In [88]:
all_granules_table = pd.DataFrame()
for table in names_of_granules_table:
    target_granules = eval(table)
    all_granules_table = pd.concat([all_granules_table,target_granules])
all_granules_table_csv = all_granules_table.to_csv(sep=',', index=False)

In [72]:
names_of_granules_table_csv = []
for name_of_granules_table in names_of_granules_table:
    names[ name_of_granules_table+'_csv'] = eval(name_of_granules_table).to_csv(sep=',', index=False)
    names_of_granules_table_csv.append(name_of_granules_table+'_csv')
names_of_granules_table_csv
granule_names = [i[:-3]+'.csv' for i in names_of_granules_table]
granules_table_csv = [eval(name_of_granules_table_csv) for name_of_granules_table_csv in names_of_granules_table_csv]

In [89]:
all_granules_table.to_csv('all_granules_table.csv',sep=',', index=False) # too large to upload to github

In [67]:
#https://stackoverflow.com/questions/61109527/how-to-load-python-dataframe-on-github-repository-as-a-csv-file
#https://stackoverflow.com/questions/50071841/how-to-push-local-files-to-github-using-python-or-post-a-commit-via-python/50072113#50072113
user = "yixiq97"
password = ##########

def updategitfiles(file_names,file_list,userid,pwd,Repo,branch,commit_message =""):
    if commit_message == "":
        commit_message = "Data Updated - "+ datetime.now().strftime('%Y-%m-%d %H:%M:%S')

    g = Github(userid,pwd)
    repo = g.get_user().get_repo(Repo)
    master_ref = repo.get_git_ref("heads/"+branch)
    master_sha = master_ref.object.sha
    base_tree = repo.get_git_tree(master_sha)
    element_list = list()
    for i in range(0,len(file_list)):
        element = InputGitTreeElement(file_names[i], '100644', 'blob', file_list[i])
        element_list.append(element)
    tree = repo.create_git_tree(element_list, base_tree)
    parent = repo.get_git_commit(master_sha)
    commit = repo.create_git_commit(commit_message, tree, [parent])
    master_ref.edit(commit.sha)
    print('Update complete')

In [11]:
if __name__ == "__main__":
    # get packagelink list from CREC collection data
    collection_json = get_CREC_packages(40)
    collection_attr = list(collection_json[0].keys())
    packageLinks = [r['packageLink'] for r in collection_json]
    collection_df = pd.DataFrame(collection_json)[['packageId','dateIssued','title','lastModified']]
    
    
    # get summary of each CERC record, in 1:1 relationship
    summary_data = [get_CREC_summary(packageLink) for packageLink in packageLinks]
    summary_attr = list(summary_data[0].keys())
    granulesLinks = [r['granulesLink'] for r in summary_data]
    record_info_df = pd.DataFrame(summary_data)[['packageId','detailsLink','pages','governmentAuthor1','category']]
    
    
    # get granule contents of each CERC record, in 1:M relationship
    granules_relation = [(granulesLink,get_CREC_granules(granulesLink)[0],get_CREC_granules(granulesLink)[1]) 
                         for granulesLink in granulesLinks]    
    granules_backward_mapping = dict([(r['granulesLink'],r['packageId']) for r in summary_data])
    names = globals()
    num_of_granules_table = 0
    names_of_granules_table = []
    for granules in granules_relation:
        gran_link = granules[0]
        pack_id = granules_backward_mapping[gran_link]
        pack_id_formatted = pack_id.replace('-','')
        names['granules_%s_df' % pack_id_formatted] = pd.DataFrame(granules[2])
        eval('granules_%s_df' % pack_id_formatted)['packageId'] = pack_id
        names_of_granules_table.append('granules_%s_df' % pack_id_formatted)
        # return to table i: eval(names_of_granules_table[i])
        num_of_granules_table += 1
    granules_count_df=pd.DataFrame(granules_relation,columns = ['granulesLink','granulesCount','json_file'])[['granulesLink','granulesCount']]
    package_granule_df = pd.DataFrame.from_dict(granules_backward_mapping,orient = 'index',columns = ['packageId'])
    package_granule_df = package_granule_df.reset_index().rename(columns = {'index':'granulesLink'})    
    
    
    # get granule contents of each CERC record
    for table in names_of_granules_table:
        title_content = []
        for link in eval(table)['granuleLink']:
            link_json = get_CREC_granuleInfo(link)
            title_content.append(txt_download(link_json))
        eval(table)['content'] = title_content
        print('granule texts collected')
    

    # Join(merge) and get original datatables
    CREC_summary = pd.merge(collection_df,record_info_df,on = 'packageId')
    CREC_summary['pages'] = CREC_summary.pages.astype('int64')
    CREC_granules =  pd.merge(pd.merge(collection_df,package_granule_df,on = 'packageId'),
                              granules_count_df,on = 'granulesLink')
    
    # update to github
    CREC_summary_csv = CREC_summary.to_csv(sep=',', index=False)
    CREC_granules_csv = CREC_granules.to_csv(sep=',', index=False)

    updategitfiles(['CREC_summary.csv','CREC_granules_summary.csv'],
                   [CREC_summary_csv,CREC_granules_csv],
                   user,password,
                   'API_data','main')
    #updategitfiles(granule_names,names_of_granules_table_csv,user,password,'API_data','main')

granule texts collected
granule texts collected
granule texts collected
granule texts collected
granule texts collected
granule texts collected
granule texts collected
granule texts collected
granule texts collected
granule texts collected
granule texts collected
granule texts collected
granule texts collected
granule texts collected
granule texts collected
granule texts collected
granule texts collected
granule texts collected
granule texts collected
granule texts collected
granule texts collected
granule texts collected
granule texts collected
granule texts collected
granule texts collected
granule texts collected
granule texts collected
granule texts collected
granule texts collected
granule texts collected
granule texts collected
granule texts collected
granule texts collected
granule texts collected
granule texts collected
granule texts collected
granule texts collected
granule texts collected
granule texts collected
granule texts collected
